## DQN Code Presentation

[23.01.27]

Minyoung Jeong

- model.py : neural network

- dqn_agent.py : define function of Agent adn replay buffer

- sir_rl.py : execute file

![dqn_algorithm](DQN_algorithm.png)

### 1. model.py

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class QNetwork(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=256, fc2_units=128):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
        """
        
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

**forward**

- state $\rightarrow$ action

- layer 1: [state size, 1] $\rightarrow$ [fc1_units, 1]   
layer 2: [fc1_units, 1] $\rightarrow$ [fc2_units, 1]   
layer 3: [fc1_units, 1] $\rightarrow$ [action size, 1]

- torch.nn:

![torch.nn](torch_nn.PNG)

<br>
- nn.Module: 원하는 신경망 모델을 만들기 위한 기초, 이를 상속받아서 QNetwork를 구성

![nn.Module](nn_Module.PNG)

<br>
- F.relu: Activation function 의 한 종류

![F.relu](RELU.PNG)


![nn](neuron.PNG)

- weight: 각 input 값에 곱해지는 비율 (민감도)

- bias: 미세 조정 값

- activation function: 기준에 따라 정보 처리 (선택, 조절)

    - ReLU(Rectified Linear Unit): $\; max\{x,0\}$

    - ELU(Exponential Linear Unit): $\; \alpha (e^x -1)\quad if\;\; x \leq 0$   
    $\; \qquad \qquad \qquad \qquad \;\;\, \qquad \quad \;\; x \qquad \;\;\; if\;\; x>0$

    [Activation function](https://en.wikipedia.org/wiki/Activation_function)

![nn.linear](nn_linear.PNG) 

In [2]:
import torch
from torchsummary import summary
from model import QNetwork

device = device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_model = QNetwork(2,2,0).to(device)
summary(test_model, (2,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 256]             768
            Linear-2                  [-1, 128]          32,896
            Linear-3                    [-1, 2]             258
Total params: 33,922
Trainable params: 33,922
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.13
Estimated Total Size (MB): 0.13
----------------------------------------------------------------


### 2. dqn_agent.py

In [3]:
import numpy as np
import random
from collections import namedtuple, deque

import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate
UPDATE_EVERY = 4        # how often to update the network

- Replay buffer: store agent's experiences

In [13]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        # deque: maxlen만큼의 정보를 양방향으로 저장할 수 있는 데이터 종류, 이 이상의 정보가 들어오면 밀려난 정보는 폐기된다.
        
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)

    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

- Agent

In [14]:
class Agent():
    """Interacts with and learns from the environment."""


    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object.

        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)
        # https://ganghee-lee.tistory.com/24

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
        

    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)

        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)
                

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.

        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        # [staes] -> [1, states]
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))
        

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states/ Set y_j to do gradient descent step
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()      # 모델 매개변수의 변화도를 재설정
        loss.backward()                 # backpropagation (prediction loss): 각 매개변수에 대한 손실(loss)의 변화도를 저장
        self.optimizer.step()           # 위의 변화도를 바탕으로 매개변수 조정 (w, b)

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU)
        

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

### 3. sir_rl.py

In [15]:
import torch
import numpy as np
from collections import deque
from scipy.integrate import odeint

- ode 작성

In [16]:
def sir(y, t, beta, gamma, u):
    S, I = y
    dydt = np.array([-beta * S * I - u * S, beta * S * I - gamma * I])
    return dydt

- SIR 환경 setting

In [32]:
class SirEnvironment:
    def __init__(self, S0=9990, I0=10):
        self.state = np.array([S0, I0])
        self.beta = 0.0001
        self.gamma = 0.5
        self.nu = 0.2
        # R_0 = 2

    def reset(self, S0=9990, I0=10):
        self.state = np.array([S0, I0])
        self.beta = 0.00015
        self.gamma = 0.5
        return self.state

    def step(self, action):
        sol = odeint(sir, self.state, np.linspace(0, 1, 101), args=(self.beta, self.gamma, self.nu*action))
        # vaccine efficiency: 20%
        
        new_state = sol[-1, :]
        S0, I0 = self.state
        S, I = new_state
        self.state = new_state
        reward = - I - action*5
        done = True if new_state[1] < 1.0 else False
        return (new_state, reward, done, 0)

In [33]:
env = SirEnvironment()
state = env.reset()
actions = []
agent = Agent(state_size=2, action_size=2, seed=0)

## Parameters
n_episodes=2000             # 2000 repeats
max_t=30                    # 30 days
eps_start=1                 # start: random
eps_end=0
eps_decay=0.99

scores = []                        # list containing scores from each episode
scores_window = deque(maxlen=100)  # last 100 scores
eps = eps_start                    # initialize epsilon

In [35]:
for i_episode in range(1, n_episodes+1):
    state = env.reset()
    score = 0
    actions = []
    for t in range(max_t):
        action = agent.act(state, eps)
        actions.append(action)
        next_state, reward, done, _ = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            break

    scores_window.append(score)       # save most recent score
    scores.append(score)              # save most recent score
    eps = max(eps_end, eps_decay*eps) # decrease epsilon
    
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
    
    if i_episode % 100 == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        print(np.array(actions)[:5], eps)
    if np.mean(scores_window)>=200.0:
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        torch.save(agent.qnetwork_local.state_dict(), 'checkpoint2.pth')
        break
    
torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')  # save model

Episode 100	Average Score: -790.41
[1 1 1 1 1] 6.821951929566876e-10
Episode 200	Average Score: -809.30
[1 1 1 1 1] 2.497055036832789e-10
Episode 300	Average Score: -799.01
[1 1 1 1 1] 9.140029014200152e-11
Episode 400	Average Score: -787.17
[1 1 1 1 1] 3.345546219372925e-11
Episode 500	Average Score: -787.36
[1 1 1 1 1] 1.2245781155148722e-11
Episode 600	Average Score: -834.90
[1 1 1 1 1] 4.4823519469386725e-12
Episode 700	Average Score: -776.21
[1 1 1 1 1] 1.6406857775485806e-12
Episode 800	Average Score: -776.05
[1 1 1 1 1] 6.005440564497957e-13
Episode 900	Average Score: -775.57
[1 1 1 1 1] 2.198185470200409e-13
Episode 1000	Average Score: -774.89
[1 1 1 1 1] 8.046069742102501e-14
Episode 1100	Average Score: -774.45
[1 1 1 1 1] 2.945121745749465e-14
Episode 1200	Average Score: -773.13
[1 1 1 1 1] 1.0780098079313767e-14
Episode 1300	Average Score: -772.30
[1 1 1 1 1] 3.9458645391262606e-15
Episode 1400	Average Score: -782.53
[1 1 1 1 1] 1.444314035603396e-15
Episode 1500	Average Sco

- visualization

In [36]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))
env2 = SirEnvironment()
state2 = env2.reset()
max_t = 30
states2 = state2
reward_sum2 = 0.
actions2 = []

for t in range(max_t):
    action2 = agent.act(state2, eps=0.0)
    actions2 = np.append(actions2, action2)
    next_state2, reward2, done, _ = env2.step(action2)
    reward_sum2 += reward2
    states2 = np.vstack((states2, next_state2))
    state2 = next_state2
    
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=list(range(max_t+1)), y=states2[:,0].flatten(), name="susceptible",
        mode='lines+markers'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=list(range(max_t+1)), y=states2[:,1].flatten(), name="infected",
        mode='lines+markers'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=list(range(max_t+1)), y=actions2, name="vaccine",
        mode='lines+markers'),
    secondary_y=True,
)
# Add figure title
fig.update_layout(
    title_text=f'{reward_sum2:.2f}: SIR model with control'
)
# Set x-axis title
fig.update_xaxes(title_text="day")
# Set y-axes titles
fig.update_yaxes(title_text="Population", secondary_y=False)
fig.update_yaxes(title_text="Vaccine", secondary_y=True)

fig.show()

In [45]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=list(range(max_t+1)), y=states2[:,1].flatten(), name="infected",
        mode='lines+markers'),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=list(range(max_t+1)), y=actions2, name="vaccine",
        mode='lines+markers'),
    secondary_y=True,
)

fig.update_layout(
    title_text=f'{reward_sum2:.2f}: SIR model with control'
)

fig.update_xaxes(title_text="day")

fig.update_yaxes(title_text="Population", secondary_y=False)
fig.update_yaxes(title_text="Vaccine", secondary_y=True)
fig.update_yaxes(range = [-10, 110], secondary_y=False)
fig.update_yaxes(range = [-0.1, 1.1], secondary_y=True)

fig.show()

In [38]:
fig2 = go.Figure(data = 
                    [go.Scatter(x=list(range(1, n_episodes+1)),
                            y=scores)]
)
fig2.update_layout(title_text='Score')
fig2.update_xaxes(title_text='number of episodes')
fig2.update_yaxes(title_text='score')

fig2.show()
